In [1]:
import flwr as fl
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.data import Dataset
import numpy as np
import os
import itertools
from tinydb import TinyDB, Query
import time

In [2]:
db = TinyDB('../Application/db.json')

In [3]:
db.all()[np.random.randint(0, len(db))]

{'number1': 0.87,
 'number2': 0.12,
 'result': 0.0,
 'time': '2023-07-06T01:54:46.846648'}

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Num GPUs Available:  0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [5]:
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [6]:
#model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
#model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

In [7]:
def data_generator():
    while(True):
        
        sample = db.all()[np.random.randint(0, len(db))]
        # our input data is an array containing 2 numbers
        X = [sample['number1'], sample['number2']]
        # our label is 1 or 0
        Y = 1 if sample['number2'] > sample['number1'] else 0
        # our generator should return the input data and the label
        yield X, [Y]
        
# create a dataset from our generator
train_dataset = tf.data.Dataset.from_generator(
    data_generator, 
    output_types = (tf.float32, tf.int32),
    output_shapes=((2), (1))
)
train_dataset = train_dataset.batch(batch_size=30)

model = Sequential([
    Input(shape=(2)),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

model.fit(
    train_dataset,
    steps_per_epoch=100,
    epochs=4
)

test_X = np.array([
    [0.1, 0.2],
    [0.3, 0.4],
    [0.5, 0.1],
    [0.7, 0.2]
])

test_Y = np.array([
    [1],
    [1],
    [0],
    [0]
])



Y = model.predict_on_batch(test_X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(Y)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 15        
                                                                 
 dense_1 (Dense)             (None, 1)                 6         
                                                                 
Total params: 21 (84.00 Byte)
Trainable params: 21 (84.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/4
100/100 [==============================] - 3s 19ms/step - loss: 0.6711 - accuracy: 0.5850
Epoch 2/4
100/100 [==============================] - 2s 18ms/step - loss: 0.6488 - accuracy: 0.6017
Epoch 3/4
100/100 [==============================] - 2s 17ms/step - loss: 0.6313 - accuracy: 0.5957
Epoch 4/4
100/100 [==============================] - 2s 18ms/step - loss: 0.6201 - accuracy: 0.5700
[[0.43]
 [0.42]
 [

In [8]:
def representative_dataset_gen():
    for _ in range(10000):
        yield [
            np.array(
                [np.random.uniform(), np.random.uniform()]
            , dtype=np.float32)
        ]

In [9]:
class CifarClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        return model.get_weights()

    def fit(self, parameters, config):
        model.set_weights(parameters)

        print("executando fit")
        model.fit(
            train_dataset,
            steps_per_epoch=100,
            epochs=4
        )

        return model.get_weights(), 2, {}

    def evaluate(self, parameters, config):
        model.set_weights(parameters)
        
        print("executando set")
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_dataset_gen
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        tflite_model = converter.convert()
        with open('model.tflite', 'wb') as f:
            f.write(tflite_model)
        
        loss, accuracy = model.evaluate(test_X, test_Y)
        time.sleep(60)
        
        return loss, len(test_X), {"accuracy": float(accuracy)}

In [10]:
fl.client.start_numpy_client(server_address="localhost:8080", client=CifarClient())

INFO flwr 2023-07-06 03:09:53,629 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-07-06 03:09:53,632 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2023-07-06 03:09:53,634 | connection.py:39 | ChannelConnectivity.READY


executando fit
Epoch 1/4
100/100 [==============================] - 2s 22ms/step - loss: 0.5927 - accuracy: 0.6003
Epoch 2/4
100/100 [==============================] - 2s 22ms/step - loss: 0.5653 - accuracy: 0.6050
Epoch 3/4
100/100 [==============================] - 2s 20ms/step - loss: 0.5388 - accuracy: 0.6160
Epoch 4/4
100/100 [==============================] - 2s 22ms/step - loss: 0.5205 - accuracy: 0.6300
executando set
INFO:tensorflow:Assets written to: C:\Users\custo\AppData\Local\Temp\tmp_7onarw3\assets


INFO:tensorflow:Assets written to: C:\Users\custo\AppData\Local\Temp\tmp_7onarw3\assets
c:\Program Files\Conda\envs\ubq2\lib\site-packages\tensorflow\lite\python\convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


1/1 [==============================] - 0s 193ms/step - loss: 0.5859 - accuracy: 0.5000
executando fit
Epoch 1/4
100/100 [==============================] - 3s 26ms/step - loss: 0.5028 - accuracy: 0.8023
Epoch 2/4
100/100 [==============================] - 2s 20ms/step - loss: 0.4743 - accuracy: 0.7970
Epoch 3/4
100/100 [==============================] - 2s 21ms/step - loss: 0.4541 - accuracy: 0.8163
Epoch 4/4
100/100 [==============================] - 2s 22ms/step - loss: 0.4275 - accuracy: 0.8307
executando set
INFO:tensorflow:Assets written to: C:\Users\custo\AppData\Local\Temp\tmpruvrs2yq\assets


INFO:tensorflow:Assets written to: C:\Users\custo\AppData\Local\Temp\tmpruvrs2yq\assets
c:\Program Files\Conda\envs\ubq2\lib\site-packages\tensorflow\lite\python\convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


1/1 [==============================] - 0s 36ms/step - loss: 0.5260 - accuracy: 0.5000
executando fit
Epoch 1/4
100/100 [==============================] - 2s 23ms/step - loss: 0.4312 - accuracy: 0.8850
Epoch 2/4
100/100 [==============================] - 3s 25ms/step - loss: 0.4107 - accuracy: 0.8887
Epoch 3/4
100/100 [==============================] - 3s 26ms/step - loss: 0.3980 - accuracy: 0.8877
Epoch 4/4
100/100 [==============================] - 2s 25ms/step - loss: 0.3835 - accuracy: 0.8963
executando set
INFO:tensorflow:Assets written to: C:\Users\custo\AppData\Local\Temp\tmp74f8lqd_\assets


INFO:tensorflow:Assets written to: C:\Users\custo\AppData\Local\Temp\tmp74f8lqd_\assets
c:\Program Files\Conda\envs\ubq2\lib\site-packages\tensorflow\lite\python\convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


1/1 [==============================] - 0s 25ms/step - loss: 0.4569 - accuracy: 0.5000


DEBUG flwr 2023-07-06 03:13:58,096 | connection.py:113 | gRPC channel closed
DEBUG:flwr:gRPC channel closed
INFO flwr 2023-07-06 03:13:58,097 | app.py:185 | Disconnect and shut down
INFO:flwr:Disconnect and shut down
